<a href="https://colab.research.google.com/github/ArvindSharma18/assAIgnment/blob/main/SFT_Book_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Reference: https://mlops.community/budget-instruction-fine-tuning-of-llama-3-8b-instructon-medical-data-with-hugging-face-google-colab-and-unsloth/

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/goodreads_library_export.csv")

In [4]:
df.columns

Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Owned Copies'],
      dtype='object')

In [6]:
df_updated = df[['Title','Author','My Rating', 'Average Rating', 'Publisher','Number of Pages', 'Year Published','Exclusive Shelf', 'My Review']]
df_updated.head()

,Title,Author,My Rating,Average Rating,Publisher,Number of Pages,Year Published,Exclusive Shelf,My Review
0,"Origin (Robert Langdon, #5)",Dan Brown,4,3.89,Doubleday,482.0,2017.0,read,NaN
1,Man's Search for Meaning,Viktor E. Frankl,5,4.37,Rider,154.0,2008.0,read,A masterful work by Viktor E Frankl who shares...
2,The Greatest Works of Khalil Gibran,Kahlil Gibran,4,4.22,Popular UK,733.0,2019.0,read,Khalil is a wonderful thinker and some of his ...
3,Carnage by Angels,Y.P. Singh,3,2.63,Shree Book Centre,382.0,2011.0,read,A fiction based on real life incidents of an I...
4,If God Was a Banker,Ravi Subramanian,3,3.33,Rupa & Co,264.0,2007.0,read,"A debut novel, high on melodrama and soap seri..."


In [7]:
df_filtered = df_updated[df_updated[['My Rating', 'My Review']].notnull().all(1)]
df_filtered

,Title,Author,My Rating,Average Rating,Publisher,Number of Pages,Year Published,Exclusive Shelf,My Review
1,Man's Search for Meaning,Viktor E. Frankl,5,4.37,Rider,154.0,2008.0,read,A masterful work by Viktor E Frankl who shares...
2,The Greatest Works of Khalil Gibran,Kahlil Gibran,4,4.22,Popular UK,733.0,2019.0,read,Khalil is a wonderful thinker and some of his ...
3,Carnage by Angels,Y.P. Singh,3,2.63,Shree Book Centre,382.0,2011.0,read,A fiction based on real life incidents of an I...
4,If God Was a Banker,Ravi Subramanian,3,3.33,Rupa & Co,264.0,2007.0,read,"A debut novel, high on melodrama and soap seri..."
5,Tamarind City: Where Modern India Began,Bishwanath Ghosh,4,3.92,Tranquebar,344.0,2012.0,read,An extended memoir of an north Indian journali...
...,...,...,...,...,...,...,...,...,...
80,Kafka on the Shore,Haruki Murakami,5,4.13,Vintage International,467.0,2006.0,read,"To be honest, I am a fan of this author for th..."
91,Crime and Punishment,Fyodor Dostoevsky,5,4.27,Penguin,671.0,2002.0,read,Wonderful Translation of a very good book! Lov...
93,How to Stop Worrying and Start Living: Time-Te...,Dale Carnegie,4,4.16,Gallery Books,358.0,2004.0,read,Idealistic.
135,Deception Point,Dan Brown,5,3.75,Pocket Books,556.0,2002.0,read,If Robert Langdon series establishes Dan Brown...


In [15]:
df_filtered=df_filtered[df_filtered['Exclusive Shelf'] == "read"]

In [16]:
def create_prompt_template(row):
  prompt = f"""<|start_header_id|>system<|end_header_id|> You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.<|eot_id|><|start_header_id|>user<|end_header_id|> Following are the Details for the Book review:\n Title: {row['Title']} \n Author: {row['Author']} \n Reviewer Rating: {row['My Rating']} \n Average Rating: {row['Average Rating']} \n Publisher: {row['Publisher']} \n Number of Pages: {row['Number of Pages']} \n Year Published: {row['Year Published']} \n Provide a Book Review Similar to your Reviwer<|eot_id|><|start_header_id|>assistant<|end_header_id|> Reviwer Review: {row['My Review']}<|eot_id|>"""

  return prompt


### Finetuning

In [1]:
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://GitHub.com/unslothai/unsloth.git"

  Cloning https://GitHub.com/unslothai/unsloth.git to /tmp/pip-install-bd9z8c18/unsloth_56cea8805c8041e1b29e3016657d07bc
  Running command git clone --filter=blob:none --quiet https://GitHub.com/unslothai/unsloth.git /tmp/pip-install-bd9z8c18/unsloth_56cea8805c8041e1b29e3016657d07bc
  Resolved https://GitHub.com/unslothai/unsloth.git to commit 64bb8cfd512a9dcd860d21563b624676f7432ec5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import json
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username":"Shekswess",
    "model_config": {
        "base_model":"unsloth/llama-3-8b-Instruct-bnb-4bit", # The base model
        "finetuned_model":"llama-3-8b-Instruct-bnb-4bit-book-review", # The fine-tuned model
        "max_seq_length": 2048, # The maximum sequence length
        "dtype":torch.float16, # The data type
        "load_in_4bit": True, # Load the model in 4-bit
    },
    "lora_config": {
      "r": 16, # The number of LoRA layers 8, 16, 32, 64
      "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"], # The target modules
      "lora_alpha":16, # The alpha value for LoRA
      "lora_dropout":0, # The dropout value for LoRA
      "bias":"none", # The bias for LoRA
      "use_gradient_checkpointing":True, # Use gradient checkpointing
      "use_rslora":False, # Use RSLora
      "use_dora":False, # Use DoRa
      "loftq_config":None # The LoFTQ configuration
    },
    "training_config": {
        "per_device_train_batch_size": 4, # The batch size
        "gradient_accumulation_steps": 4, # The gradient accumulation steps
        "warmup_steps": 5, # The warmup steps
        "max_steps":0, # The maximum steps (0 if the epochs are defined)
        "num_train_epochs": 6, # The number of training epochs(0 if the maximum steps are defined)
        "learning_rate": 2e-3, # The learning rate
        "fp16": not torch.cuda.is_bf16_supported(), # The fp16
        "bf16": torch.cuda.is_bf16_supported(), # The bf16
        "logging_steps": 1, # The logging steps
        "optim" :"adamw_8bit", # The optimizer
        "weight_decay" : 0.015,  # The weight decay
        "lr_scheduler_type": "linear", # The learning rate scheduler
        "seed" : 42, # The seed
        "output_dir" : "outputs", # The output directory
    }
}

In [3]:
dataset_train = load_dataset("csv", data_files="/content/My_Book_Reviews_Dataset.csv", split = "train")

In [4]:
dataset_train

Dataset({
    features: ['Unnamed: 0', 'Title', 'Author', 'My Rating', 'Average Rating', 'Publisher', 'Number of Pages', 'Year Published', 'Exclusive Shelf', 'My Review', 'Prompt_Instruction'],
    num_rows: 64
})

In [5]:
# Loading the model and the tokinizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config.get("model_config").get("base_model"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dtype = config.get("model_config").get("dtype"),
    load_in_4bit = config.get("model_config").get("load_in_4bit"),
)

# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "Prompt_Instruction",
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [7]:
# Memory statistics before training
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")

Reserved Memory: 5.59GB
Max Memory: 14.75GB


In [8]:
# Training the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 64 | Num Epochs = 6
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 24
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.688300
2,3.440100
3,3.029300
4,2.942100
5,2.410300
6,2.169700
7,2.064000
8,1.904800
9,1.704400
10,2.092800


In [9]:
# Memory statistics after training
used_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
used_memory_lora = round(used_memory - reserved_memory, 2)
used_memory_persentage = round((used_memory / max_memory) * 100, 2)
used_memory_lora_persentage = round((used_memory_lora / max_memory) * 100, 2)
print(f"Used Memory: {used_memory}GB ({used_memory_persentage}%)")
print(f"Used Memory for training(fine-tuning) LoRA: {used_memory_lora}GB ({used_memory_lora_persentage}%)")

Used Memory: 12.66GB (85.83%)
Used Memory for training(fine-tuning) LoRA: 7.07GB (47.93%)


In [20]:
FastLanguageModel.for_inference(model)

# Tokenizing the input and generating the output
inputs = tokenizer(
[
    "<|start_header_id|>system<|end_header_id|> You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.<|eot_id|><|start_header_id|>user<|end_header_id|> Following are the Details for the Book review:\n Title: Murder on the Orient Express (Hercule Poirot, #10)\n Author: Agatha Christie \n Reviewer Rating: 5 \n Average Rating: 4.2 \n Publisher: HarperCollins \n Number of Pages: 274.0 \n Year Published: 2007.0 \n Provide a Book Review Similar to your Reviwer<|eot_id|>",
    "<|start_header_id|>system<|end_header_id|> You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.<|eot_id|><|start_header_id|>user<|end_header_id|> Following are the Details for the Book review:\n Title: To Kill a Mockingbird \n Author: Harper Lee \n Reviewer Rating: 5 \n Average Rating: 4.26 \n Publisher: Harper Perennial Modern Classics  \n Number of Pages: 323.0 \n Year Published: 2006.0 \n Provide a Book Review Similar to your Reviwer<|eot_id|>"
], return_tensors = "pt", padding= True).to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 1024)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["system You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.user Following are the Details for the Book review:\n Title: Murder on the Orient Express (Hercule Poirot, #10)\n Author: Agatha Christie \n Reviewer Rating: 5 \n Average Rating: 4.2 \n Publisher: HarperCollins \n Number of Pages: 274.0 \n Year Published: 2007.0 \n Provide a Book Review Similar to your Reviwerassistant Reviwer Review: Excellent story. Very good plot. Great ending. What more could you ask for?",
 "system You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.user Following are the Details for the Book review:\n Title: To Kill a Mockingbird \n Author: Harper Lee \n Reviewer Rating: 5 \n Average Rating: 4.26 \n Publisher: Harper Perennial Modern Clas

In [16]:
notebook_login()

In [21]:
model.save_pretrained_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_16bit")


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.82 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:33<00:00,  2.93s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00001-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00002-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00003-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00004-of-00004.bin...
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.88 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:25<00:00,  4.54s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00001-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00002-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00003-of-00004.bin...
Unsloth: Saving llama-3-8b-Instruct-bnb-4bit-book-review/pytorch_model-00004-of-00004.bin...


...

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/llama-3-8b-Instruct-bnb-4bit-book-review
